In [173]:
%pip install geopy
%pip install pandas
%pip install pulp
%pip install itertools


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [176]:
import pandas as pd
import numpy as np
import time
#from itertools import combinations
import geopy.distance
import pulp
import random
import numpy as np

#import itertools

In [177]:
df = pd.read_csv('datasets_round_1/part_b_input_dataset_2.csv')
df

,order_id,lng,lat,depot_lat,depot_lng
0,2556635,126.53566,43.82912,43.8502,126.5445
1,1507929,126.54484,43.84685,43.8502,126.5445
2,2234840,126.51653,43.81547,43.8502,126.5445
3,511292,126.56568,43.84987,43.8502,126.5445
4,3846410,126.55650,43.84634,43.8502,126.5445
5,3731521,126.54651,43.84220,43.8502,126.5445
6,720049,126.55835,43.85397,43.8502,126.5445
7,410698,126.52634,43.81835,43.8502,126.5445
8,3806465,126.52507,43.82184,43.8502,126.5445
9,3619717,126.49256,43.81104,43.8502,126.5445


In [178]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # radius of Earth in kilometers
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    res = R * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 2)

In [179]:
def get_req_number_of_nodes():
    tuples = df[['lat','lng']].to_records(index=False)
    list_of_tuples = list(tuples)
    depot_coords = (df[['depot_lat','depot_lng']].drop_duplicates()['depot_lat'][0], df[['depot_lat','depot_lng']].drop_duplicates()['depot_lng'][0])
    list_of_tuples.insert(0, depot_coords)
    return list_of_tuples

In [180]:
def get_cost_matrix(list_of_tuples):
    dist_adj_matrix = np.zeros((len(list_of_tuples),len(list_of_tuples)))
    for i in range(len(list_of_tuples)):
        for j in range(len(list_of_tuples)):
            if i == j:
                dist_adj_matrix[i][j] = 0 # 10^7 as max value
            else:
                lat_i = list_of_tuples[i][0]
                lng_i = list_of_tuples[i][1]
                coords_1 = (lat_i,lng_i)
                lat_j = list_of_tuples[j][0]
                lng_j = list_of_tuples[j][1]
                coords_2 = (lat_j,lng_j)
                # dist = haversine(lat_i,lng_i,lat_j,lng_j)
                dist = geopy.distance.geodesic(coords_1, coords_2).km
                # print(i,j,dist)
                dist_adj_matrix[i][j] = dist
            
    return dist_adj_matrix

In [181]:
list_of_tuples = get_req_number_of_nodes()
print('Number of tuples {}'.format(len(list_of_tuples)))
dist_adj_matrix = get_cost_matrix(list_of_tuples).round(2)

Number of tuples 51


In [182]:
list_of_tuples

[(43.8502, 126.5445),
 (43.82912, 126.53566),
 (43.84685, 126.54484),
 (43.81547, 126.51653),
 (43.84987, 126.56568),
 (43.84634, 126.5565),
 (43.8422, 126.54651),
 (43.85397, 126.55835),
 (43.81835, 126.52634),
 (43.82184, 126.52507),
 (43.81104, 126.49256),
 (43.83859, 126.52478),
 (43.81664, 126.51849),
 (43.83586, 126.53544),
 (43.82793, 126.53486),
 (43.84308, 126.54089),
 (43.85554, 126.5628),
 (43.8274, 126.52712),
 (43.8472, 126.5552),
 (43.82267, 126.52799),
 (43.84056, 126.54469),
 (43.80999, 126.50244),
 (43.84127, 126.55913),
 (43.82474, 126.51983),
 (43.82592, 126.51028),
 (43.82348, 126.51366),
 (43.84821, 126.55501),
 (43.84739, 126.55891),
 (43.82733, 126.52523),
 (43.82125, 126.51461),
 (43.83759, 126.54318),
 (43.81535, 126.51441),
 (43.81582, 126.49753),
 (43.8452, 126.55553),
 (43.8412, 126.54036),
 (43.81575, 126.51801),
 (43.85006, 126.55724),
 (43.84231, 126.56152),
 (43.82286, 126.52302),
 (43.82199, 126.52705),
 (43.82484, 126.52391),
 (43.84589, 126.56581),
 (

In [183]:
dist_adj_matrix

array([[0.  , 2.45, 0.37, ..., 1.78, 3.69, 1.29],
       [2.45, 0.  , 2.1 , ..., 3.7 , 1.35, 1.67],
       [0.37, 2.1 , 0.  , ..., 1.88, 3.38, 0.95],
       ...,
       [1.78, 3.7 , 1.88, ..., 0.  , 5.05, 2.05],
       [3.69, 1.35, 3.38, ..., 5.05, 0.  , 3.01],
       [1.29, 1.67, 0.95, ..., 2.05, 3.01, 0.  ]])

In [184]:
dist_adj_matrix.shape

(51, 51)

In [185]:
n = dist_adj_matrix.shape[0]
requests = list(range(1,n))
max_nvans = n-1
capacity_per_van = 20
fixed_cost_per_van = 0

In [186]:
def calculate_total_distance(route, dist_matrix):
    """
    Calculate the total distance of a given route using the distance matrix.
    """
    total_distance = 0
    num_points = len(route)

    for i in range(num_points-1):
        current_node = route[i]
        next_node = route[i + 1]
        total_distance += dist_matrix[current_node, next_node]

    return total_distance + fixed_cost_per_van*int(len(route)>2)

In [187]:
def init_allocation(max_nvans):
    routes = []
    for i in range(1,max_nvans+1):
        route = [0,i,0]
        routes.append(route)
    return routes

In [188]:
routes = init_allocation(max_nvans)

In [189]:
routes

[[0, 1, 0],
 [0, 2, 0],
 [0, 3, 0],
 [0, 4, 0],
 [0, 5, 0],
 [0, 6, 0],
 [0, 7, 0],
 [0, 8, 0],
 [0, 9, 0],
 [0, 10, 0],
 [0, 11, 0],
 [0, 12, 0],
 [0, 13, 0],
 [0, 14, 0],
 [0, 15, 0],
 [0, 16, 0],
 [0, 17, 0],
 [0, 18, 0],
 [0, 19, 0],
 [0, 20, 0],
 [0, 21, 0],
 [0, 22, 0],
 [0, 23, 0],
 [0, 24, 0],
 [0, 25, 0],
 [0, 26, 0],
 [0, 27, 0],
 [0, 28, 0],
 [0, 29, 0],
 [0, 30, 0],
 [0, 31, 0],
 [0, 32, 0],
 [0, 33, 0],
 [0, 34, 0],
 [0, 35, 0],
 [0, 36, 0],
 [0, 37, 0],
 [0, 38, 0],
 [0, 39, 0],
 [0, 40, 0],
 [0, 41, 0],
 [0, 42, 0],
 [0, 43, 0],
 [0, 44, 0],
 [0, 45, 0],
 [0, 46, 0],
 [0, 47, 0],
 [0, 48, 0],
 [0, 49, 0],
 [0, 50, 0]]

In [190]:
def calculate_vrp_cost(routes,dist_matrix):
    cost = 0
    for route in routes:
        cost += calculate_total_distance(route, dist_matrix)
    return cost
        

In [191]:
total_cost = calculate_vrp_cost(routes,dist_adj_matrix)

In [192]:
total_cost

253.56000000000006

In [193]:
class destroy:
    def __init__(self):
        self.w = [0.5,0.5]

    def worst_removal(self, routes, dist_matrix):
        max_cost_request = -1
        max_cost_request_cost = 0
        max_cost_route_num = -1
        route_num = -1
        
        for route in routes:
            route_num += 1 
            init_cost = calculate_total_distance(route, dist_matrix)
            #print(route_num,init_cost)
            if len(route) > 2:
                for i in range(1,len(route)-1):
                    new_route = route[:i] + route[i+1:]
                    #print(new_route)
                    cost = calculate_total_distance(new_route, dist_matrix)
                    #print('-----------',cost,(init_cost-cost),max_cost_request_cost,' max_cost_route: ',max_cost_route_num,\
                    #      ' route_num: ',route_num,' position in route ',i)
                    if (init_cost-cost) > max_cost_request_cost:
                        max_cost_request_cost = (init_cost-cost)
                        max_cost_request = i
                        max_cost_route_num = route_num

        request = routes[max_cost_route_num][max_cost_request]
        routes[max_cost_route_num] = routes[max_cost_route_num][:max_cost_request] + routes[max_cost_route_num][max_cost_request+1:]
        return request, routes

    def random_removal(self,routes,requests):
        request = random.choice(requests)
        route_num=-1
        destroy_route = -1
        for route in routes:
            route_num+=1
            if request in route:
                destroy_route = route_num
                break
        idx = routes[destroy_route].index(request)
        routes[destroy_route] = routes[destroy_route][:idx] + routes[destroy_route][idx+1:]
        return request, routes                          

In [194]:
class repair:
    def __init__(self):
        self.w = [1,1]
        
    def greedy_insertion(self, request, routes, dist_matrix):
        min_cost_request_cost = 10000000
        min_cost_route_num = -1
        min_cost_route_pos = -1
        route_num = -1
        for route in routes:
            route_num += 1 
            if len(route) < capacity_per_van:
                init_cost = calculate_total_distance(route, dist_matrix)
                for pos in range(1,len(route)):
                    new_route = route[:pos]+[request]+route[pos:]
                    cost = calculate_total_distance(new_route, dist_matrix)
                    delta_cost = cost-init_cost
                    if delta_cost < min_cost_request_cost:
                        min_cost_request_cost = delta_cost
                        min_cost_route_num = route_num
                        min_cost_route_pos = pos

        routes[min_cost_route_num] = routes[min_cost_route_num][:min_cost_route_pos]+\
        [request]+routes[min_cost_route_num][min_cost_route_pos:]
        return routes
        

In [195]:
d = destroy()
r = repair()
request, droutes = d.worst_removal(routes,dist_adj_matrix)
rroutes = r.greedy_insertion(request, droutes,dist_adj_matrix)

In [197]:
#simulated annealing
choices = [0,1]
cost = calculate_vrp_cost(routes,dist_adj_matrix)
ntrial = 1
T = 30
alpha = 0.99
rho = 0.01
d.w = [0.5,0.5]
while ntrial <= 10000:
    choice = np.random.choice(choices, 1, p=d.w)[0]
    old_routes = routes.copy()
    if choice == 0:
        req, routes = d.worst_removal(routes,dist_adj_matrix)
    else:
        req, routes = d.random_removal(routes,requests)
        
    routes = r.greedy_insertion(req, routes,dist_adj_matrix)
    new_cost = calculate_vrp_cost(routes,dist_adj_matrix)
    if new_cost > cost:
        if np.random.rand() > np.exp(-(new_cost - cost)/T):
            routes = old_routes
    #print(d.w,choice)
    d.w[choice] = (1-rho)*d.w[choice] + (cost-new_cost)/cost*rho
    d.w[0] = d.w[0]/(d.w[0]+d.w[1])
    d.w[1] = 1-d.w[0]
    #print(d.w)
    T = alpha*T
    ntrial += 1

In [198]:
routes

[[0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 15, 44, 34, 13, 47, 17, 40, 38, 9, 49, 8, 39, 19, 14, 1, 30, 20, 43, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 36, 27, 41, 4, 48, 16, 7, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 11, 24, 32, 10, 21, 31, 3, 35, 12, 45, 46, 29, 25, 23, 28, 42, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 2, 6, 50, 22, 37, 33, 5, 18, 26, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0]]